## Genome quality control and selection
All candidate genomes were evaluated using a quality control procedure based on assembly metrics. For each assembly, a composite quality score (`quality_score`) was calculated by integrating:
- Assembly continuity (N50)
- Sequencing depth (coverage)
- Assembly fragmentation (number of contigs ≥200 bp)

Genomes were then grouped by geographic region. Within each region, the five assemblies with the highest quality scores were selected. This stratified sampling strategy was used to:
- Maximize the overall quality of the final dataset
- Preserve regional representativeness
- Avoid bias caused by over-representation of regions with higher numbers of available isolates

Only the top-ranked genomes per region, together with their associated metadata and quality metrics, were retained for downstream analyses.

In [4]:
import pandas as pd
import re
from pathlib import Path


In [5]:
path_df = Path('/salmonella-pangenome-SP/data/Metadata/meta_sp_total.txt')
assert path_df.exists(), f'no se encuentra el fichero'

In [6]:
df = pd.read_csv(path_df, sep='\t', dtype=str, engine='python')
print(df.head(3))
print('original columns', list(df.columns))
print('Shape: ', df.shape)

     Uberstrain        Name  \
0  SAL_BA6133AA   OSF031120   
1  SAL_DA9157AA  EC20100134   
2  SAL_DA9158AA  EC20100130   

  Data Source(Accession No.;Sequencing Platform;Sequencing Library;Insert Size;Experiment;Bases;Average Length;Status)  \
0  SRR1995504;ILLUMINA;Paired;500;SRX1008539;2371...                                                                     
1  SRR1183994;LS454;Single;;SRX481517;;;NEW,GCF_0...                                                                     
2  SRR1183993;LS454;Single;;SRX481516;;;NEW,GCF_0...                                                                     

        Barcode Source Niche Source Type Source Details Collection Year  \
0  SAL_BA6133AA      Poultry     Poultry        chicken            2014   
1  SAL_DA9157AA      Poultry     Poultry        chicken            2009   
2  SAL_DA9158AA      Poultry     Poultry        chicken            2009   

  Collection Month Collection Day  ... Release Date     Status Coverage  \
0         

## Normalize columns downloaded from Enterobase

In [7]:
def normalize_col(col: str) -> str:
    c = str(col).strip()    
    c = re.sub(r"[^A-Za-z0-9]+", "_", c)
    c = c.replace("_200_bp", "")
    c = re.sub(r"_+", "_", c)                 
    return c.strip("_").lower()

df.columns = [normalize_col(c) for c in df.columns]
print("Normalize columns:", list(df.columns))


Normalize columns: ['uberstrain', 'name', 'data_source_accession_no_sequencing_platform_sequencing_library_insert_size_experiment_bases_average_length_status', 'barcode', 'source_niche', 'source_type', 'source_details', 'collection_year', 'collection_month', 'collection_day', 'collection_time', 'continent', 'country', 'region', 'district', 'city', 'post_code', 'latitude', 'longitude', 'serovar', 'subspecies', 'disease', 'antigenic_formulas', 'lab_contact', 'phage_type', 'comment', 'bio_project_id', 'project_id', 'sample_id', 'secondary_sample_id', 'date_entered', 'release_date', 'status', 'coverage', 'n50', 'length', 'species', 'contig_number', 'low_quality_bases', 'version', 'assembly_barcode']


## Identify and split composite columns

In [8]:
column_c = [c for c in df.columns if 'data_source_' in c]
expanded = df[column_c[0]].str.split(';', n=7, expand=True)
expanded.columns = ['accession_no',
                    'sequencing_platform',
                    'sequencing_library',
                    'insert_size',
                    'experiment',
                    'bases',
                    'average_length',
                    'status'
                   ]  
for col in expanded.columns: 
    df[col] = expanded[col].str.strip() 

# Remove the original column

df = df.drop(columns=[column_c[0]])
print(df.head(2))



     uberstrain        name       barcode source_niche source_type  \
0  SAL_BA6133AA   OSF031120  SAL_BA6133AA      Poultry     Poultry   
1  SAL_DA9157AA  EC20100134  SAL_DA9157AA      Poultry     Poultry   

  source_details collection_year collection_month collection_day  \
0        chicken            2014                4            NaN   
1        chicken            2009              NaN            NaN   

  collection_time  ... low_quality_bases version assembly_barcode  \
0             NaN  ...             19814     2.2  SAL_EA5295AA_AS   
1             NaN  ...                 8     NaN  SAL_KA3434AA_AS   

  accession_no sequencing_platform sequencing_library insert_size  experiment  \
0   SRR1995504            ILLUMINA             Paired         500  SRX1008539   
1   SRR1183994               LS454             Single               SRX481517   

       bases average_length  
0  237155400            225  
1                            

[2 rows x 47 columns]


In [16]:
# Convert the variables of interest to numeric values
for col in ['n50', 'coverage', 'contig_number']:
    df[col]= pd.to_numeric(df[col], errors='coerce')


In [18]:
# Creating quality metrics
df['quality_score']= df['n50'].fillna(0)/500000 + df['coverage'].fillna(0)/50 + (200 - df['contig_number'].fillna(200))/20
# It's divided so that each metric has a similar weight. If we don't know how many contigs that entry has, 
# we penalize it heavily, assuming a gigantic value, so that sample will come out with a very low score and will almost never be the best.

print(df.head(5))

# Working just with poultry sources
df_trabajo = df[df['source_niche']=='Poultry']

print(df_trabajo.head(5))

     uberstrain        name       barcode source_niche source_type  \
0  SAL_BA6133AA   OSF031120  SAL_BA6133AA      Poultry     Poultry   
1  SAL_DA9157AA  EC20100134  SAL_DA9157AA      Poultry     Poultry   
2  SAL_DA9158AA  EC20100130  SAL_DA9158AA      Poultry     Poultry   
3  SAL_DA9159AA  EC20090884  SAL_DA9159AA      Poultry     Poultry   
4  SAL_DA9160AA  EC20090531  SAL_DA9160AA      Poultry     Poultry   

  source_details collection_year collection_month collection_day  \
0        chicken            2014                4            NaN   
1        chicken            2009              NaN            NaN   
2        chicken            2009              NaN            NaN   
3        chicken            2009              NaN            NaN   
4        chicken            2009              NaN            NaN   

  collection_time  ... version assembly_barcode accession_no  \
0             NaN  ...     2.2  SAL_EA5295AA_AS   SRR1995504   
1             NaN  ...     NaN  SAL_KA3434

## Working dataframe

In [26]:
# Creating metadata with the main data of the genomes to be worked on
df_final = (
    df_trabajo
    .groupby('region', group_keys=False)
    .apply(lambda x: x.nlargest(5, 'quality_score'))
    .assign(
        # 1. FASTA name per genome
        sample_fasta=lambda d: (
            d['region'].astype(str) + "_" +
            d['name'].astype(str) + "_" +
            d['accession_no'].astype(str)
        ),

        # 2. Sequence type
        sequence_type=lambda d: np.where(
            d['serovar'].str.lower().str.contains("enteritidis"),
            "ST11",
            np.where(
                d['serovar'].str.lower().str.contains("heidelberg"),
                "ST15",
                np.nan
            )
        ),

        # 3. FASTA assembly
        fasta_source=lambda d: np.where(
            d['accession_no'].str.startswith("GCF"),
            "NCBI_RefSeq",
            np.where(
                d['accession_no'].str.startswith("SRR"),
                "EnteroBase_assembly",
                np.nan
            )
        )
    )
    [[
        "region", "name", "accession_no", "assembly_barcode",
        "serovar", "sequence_type",
        "source_niche", "source_details",
        "collection_year", "country", "lab_contact",
        "n50", "contig_number", "coverage", "quality_score",
        "sample_fasta", "fasta_source"
    ]]
)

df_final['sample_fasta'] = df_final['sample_fasta'].str.replace(" ", "_", regex=False)

df_final[df_final['sample_fasta'].str.contains(" ")]

print(df_final)



              region        name   accession_no assembly_barcode      serovar  \
11           Alberta  EC20090641     SRR1183981  SAL_KA3404AA_AS  Enteritidis   
35           Alberta  EC20111515  GCF_000624315  SAL_PA7468AA_AS  Enteritidis   
32           Alberta  EC20120677  GCF_000625755  SAL_PA7476AA_AS  Enteritidis   
34           Alberta  EC20111514  GCF_000624295  SAL_PA7505AA_AS  Enteritidis   
10           Alberta  EC20090698     SRR1183982  SAL_KA3468AA_AS  Enteritidis   
31  British Columbia  EC20120765  GCF_000624995  SAL_PA7576AA_AS  Enteritidis   
30  British Columbia  EC20111554  GCF_000624335  SAL_PA7506AA_AS  Enteritidis   
33  British Columbia  EC20111510  GCF_000626555  SAL_PA7485AA_AS  Enteritidis   
17  British Columbia  EC20120686  GCF_000625555  SAL_PA7535AA_AS  Enteritidis   
37  British Columbia  EC20120685  GCF_000625535  SAL_PA7470AA_AS  Enteritidis   
46     New Brunswick   N13-02934     SRR5241846  SAL_QA6543AA_AS   Heidelberg   
68     New Brunswick   N13-0

/tmp/ipykernel_5250/1706637055.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(5, 'quality_score'))


In [27]:
out_path = "/salmonella-pangenome-SP/data/Metadata/metadata_samples.tsv"

df_final.to_csv(
    out_path,
    sep="\t",
    index=False
)


In [30]:
genomes_df = df_final[["sample_fasta", "accession_no", "fasta_source"]]

out_path = "/salmonella-pangenome-SP/data/Metadata/genomes_manifest.tsv"
genomes_df.to_csv(
    out_path,
    sep="\t",
    index=False
)